In [5]:
import lyricsgenius
from multiprocessing import current_process, Pool
import time
import sqlite3

## Genius API

In [6]:
genius_tokens = [
    "tETKVL7G-wtix9PAAPxsDeW-j8sS1D4L-zyWei7n1Xq1WUh9VuwFLHAXlBP5X5zq",
    "6rUGP-maK090aBdaNi1Z0qm2CtFDoIyteABaTh441aYc4eKHyKYPv5Q8luozmFC0",
    "whDHSeFHAeKA9f3mEt63XMBJVBBp-OO-01qRzn4mu8L1pQoI5hFywcm5MK2DkxgV",
    "Cj82uimfARNlK36SjqkeXAqpqoWdACXWwm1kIRHzYLAbkUiHYcvNqfj9Iq99J8iR"
]
genius_apis = []
for token in genius_tokens:
    genius_apis.append(lyricsgenius.Genius(token))

In [7]:
def get_lyrics(song, artist):
    try:
        proc = current_process()._identity[0]
    except:
        proc = 0
        
    return genius_apis[proc%len(genius_tokens)].search_song(song, artist)

In [8]:
def get_lyrics_serial(song_artists):
    start = time.perf_counter()

    genius_songs = []
    for song_artist in song_artists:
        genius_songs.append(get_lyrics(song_artist[0], song_artist[1]))

    print(f"{time.perf_counter()-start} seconds for {len(song_artists)} songs")
    
    return genius_songs

In [9]:
def get_lyrics_parallel(song_artists):
    start = time.perf_counter()

    with Pool() as p:
        genius_songs = p.starmap(get_lyrics, song_artists)

    print(f"{time.perf_counter()-start} seconds for {len(song_artists)} songs")
    
    return genius_songs

In [10]:
def print_num_with_lyrics(songs):
    num_lyrics = 0
    for i in range(len(songs)):
        if songs[i] != None:
            num_lyrics += 1
    print(f"{num_lyrics}/{len(songs)} songs have lyrics")

In [11]:
song_artists_sample = [
    ["The Magician", "Andy Shauf"],
    ["Begin Again", "Andy Shauf"],
    ["Martha Sways", "Andy Shauf"],
    ["I Will", "The Beatles"],
    ["90210", "Travis Scott"],
    ["Soma", "The Strokes"],
    ["Hey Jude", "The Beatles"],
    ["Hey Bulldog", "The Beatles"],
    ["Self Care", "Mac Miller"],
    ["Machu Picchu", "The Strokes"],
    ["Blue Boy", "Mac DeMarco"],
    ["Dreams Via Memories", "Ceramic Animal"],
    ["White Gloves", "Khruangbin"],
    ["Down the Line", "Beach Fossils"],
    ["Sleep Apnea", "Beach Fossils"]
]

## SQLite

In [12]:
# Create connection cursor
connection = sqlite3.connect("../backend/spotify_table.db")
cursor = connection.cursor()

In [13]:
# Create lyrics table
cursor.execute("DROP TABLE Lyrics")
cursor.execute("""  CREATE TABLE Lyrics AS
                        SELECT track_id
                        FROM spotify_table; """)
cursor.execute("""  ALTER TABLE Lyrics
                    ADD lyrics TEXT; """)

In [14]:
def query_unsearched_songs():
    cursor.execute("""  SELECT track_name, artist_name, track_id
                        FROM spotify_table
                            NATURAL JOIN Lyrics
                        WHERE lyrics IS NULL
                        LIMIT 25; """) # songs that we haven't searched lyrics for yet
    song_artists = cursor.fetchall()
#     for s_a in song_artists: print(s_a[0], s_a[1])
    return song_artists

In [17]:
def insert_lyrics(lyrics, track_ids):
    cursor.executemany("""  UPDATE Lyrics SET
                                lyrics=?
                            WHERE track_id=?; """,
                       zip(lyrics, track_ids));

## Main Loop

In [18]:
songs = ["trash"]
while songs:
    songs = query_unsearched_songs()
    track_names, artist_names, track_ids = zip(*songs)
    g_songs = get_lyrics_parallel(list(zip(track_names, artist_names)))
    lyrics = [g_song.lyrics if g_song!=None else "" for g_song in g_songs]
    insert_lyrics(lyrics, track_ids)
    connection.commit()

Searching for "Don't Let Me Be Lonely Tonight" by Joseph Williams...Searching for "Perdu d'avance (par Gad Elmaleh)" by Martin & les fées...Searching for "C'est beau de faire un Show" by Henri Salvador...


Searching for "Dis-moi Monsieur Gordon Cooper" by Henri Salvador...
No results found for: 'Perdu d'avance (par Gad Elmaleh) Martin & les fées'
No results found for: 'C'est beau de faire un Show Henri Salvador'No results found for: 'Dis-moi Monsieur Gordon Cooper Henri Salvador'Searching for "Ouverture" by Fabien Nataf...


Searching for "Le petit souper aux chandelles" by Henri Salvador...
Searching for "Premières recherches (par Paul Ventimila, Lorie Pester, Véronique Jannot, Michèle Laroque & Gérard Lenorman)" by Martin & les fées...
No results found for: 'Ouverture Fabien Nataf'
Searching for "Let Me Let Go" by Laura Mayne...
No results found for: 'Premières recherches (par Paul Ventimila, Lorie Pester, Véronique Jannot, Michèle Laroque & Gérard Lenorman) Martin & les fées'
Searc

In [21]:
# cursor.execute("""  SELECT track_name, artist_name
#                     FROM spotify_table
#                         NATURAL JOIN Lyrics
#                     WHERE lyrics IS NOT NULL
#                         AND lyrics<>''; """)
# foo = cursor.fetchall()
# print(len(foo))

Don't Let Me Be Lonely Tonight Joseph Williams
Le petit souper aux chandelles Henri Salvador
Let Me Let Go Laura Mayne
Helka Chorus
J'ai demandé à la lune Idoles De La Musique
Diane Jean Claude Corbel
Ultra Man 80 Bernard Minet
Ladies And Gentlemen Amalya
Clic clac cloc Dorothée
Pourquoi aller plus loin ? Jean Claude Corbel
Quand je monte chez toi Henri Salvador
Comme minuit sonnait Jean Piat
Le Fêtard Henri Salvador
Le mariage des poupées Lucie Dolene
Une chanson douce Henri Salvador
A ty się śmiejesz ze mnie Chorus
C Est Le Be Bop Henri Salvador
Silence on tourne Henri Salvador
For the Game Chorus
Sailing, Sailing Larry Groce
20


In [23]:
connection.close()